In [1]:
import polars as pl
import numpy as np
from tqdm import tqdm
from scipy.sparse import csr_matrix
import implicit

RANDOM_STATE = 42
N_PREDICTIONS = 100


/home/user/conda/envs/anpalmak/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test =  pl.read_parquet('/home/jovyan/shares/SR004.nfs2/amaksimova/exp/hh_recsys_test_hh.pq')
test = test.to_pandas()

In [3]:
import pandas as pd
near = pd.read_json("/home/jovyan/shares/SR004.nfs2/amaksimova/exp/30/nearest50_names.jsonl",lines=True)

In [8]:
vacancy_id = test['vacancy_id']
action_type = test['action_type']

In [6]:
d = {}
q = list(near['query'])
p = list(near['pos'])
for i in range(len(q)):
    d[q[i]] = p[i]
d

{'v_2338678': ['v_777257',
  'v_1491944',
  'v_1347392',
  'v_1207393',
  'v_2044588',
  'v_675834',
  'v_2602676',
  'v_272351',
  'v_1823193',
  'v_2355698',
  'v_1126263',
  'v_2585584',
  'v_1416573',
  'v_2208440',
  'v_1396022',
  'v_971332',
  'v_1691017',
  'v_2262078',
  'v_2393074',
  'v_1394365',
  'v_1818840',
  'v_2549070',
  'v_1185704'],
 'v_2207196': ['v_2208440',
  'v_1691017',
  'v_1416573',
  'v_1394365',
  'v_777257',
  'v_1823193',
  'v_2549070',
  'v_2585584',
  'v_1396022',
  'v_1126263',
  'v_971332',
  'v_272351',
  'v_675834',
  'v_1347392',
  'v_2262078',
  'v_1491944',
  'v_1185704',
  'v_1207393',
  'v_2393074',
  'v_2355698',
  'v_2044588',
  'v_1818840',
  'v_2602676'],
 'v_1753940': ['v_1491944',
  'v_2393074',
  'v_2602676',
  'v_971332',
  'v_272351',
  'v_1396022',
  'v_2549070',
  'v_1126263',
  'v_675834',
  'v_1823193',
  'v_777257',
  'v_1691017',
  'v_1185704',
  'v_2355698',
  'v_1207393',
  'v_1347392',
  'v_1818840',
  'v_2208440',
  'v_141657

In [9]:
from tqdm import tqdm
recs = []
for i in range(len(action_type)):
    rec = []
    for j in range(len(action_type[i])):
        if action_type[i][j]!=2:
            rec.extend(d[vacancy_id[i][j]])
    recs.append(rec)

In [16]:
recs_c = []
from collections import Counter

def top_n_frequent_elements(arr, n):
    counts = Counter(arr)
    return [item for item, _ in counts.most_common(n)]



import numpy as np
for i in tqdm(range(len(recs))):
    a = recs[i]
    if len(a)>30:
        recs_c.append(top_n_frequent_elements(a, 30))
    else:
        recs_c.append(a)
    

  0%|          | 0/83189 [00:00<?, ?it/s]

100%|██████████| 83189/83189 [00:00<00:00, 100435.07it/s]


In [17]:
d = pd.DataFrame()
d['user_id'] = test['user_id']
d['vacancy_id'] = recs_c
d

,user_id,vacancy_id
0,u_482520,"[v_1006605, v_2546271, v_949577, v_2421687, v_..."
1,u_582132,"[v_105810, v_114546, v_2017388, v_458496, v_22..."
2,u_212584,"[v_266593, v_1934251, v_1834410, v_1074425, v_..."
3,u_425177,[]
4,u_700997,"[v_1444723, v_864897, v_858969, v_867642, v_19..."
...,...,...
83184,u_499368,"[v_1919599, v_1791706, v_2199307, v_449144, v_..."
83185,u_1084868,[]
83186,u_1138032,"[v_498447, v_894010, v_1871770, v_351246, v_11..."
83187,u_608102,"[v_451255, v_200853, v_1348821, v_1428973, v_7..."


In [ ]:
d.to_json('/home/jovyan/shares/SR004.nfs2/amaksimova/exp/30/test_res.jsonl', orient='records', lines=True, force_ascii=False)